In [42]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder

In [9]:
df = pd.read_csv('./data/train.csv')
df.head()
print(df.shape)

(3960, 82)


In [10]:
df['PAQ-PAQ_Total'] = df['PAQ_A-PAQ_A_Total'].combine_first(df['PAQ_C-PAQ_C_Total'])
df = df.drop(['PAQ_A-PAQ_A_Total','PAQ_C-PAQ_C_Total'],axis=1)
df.columns

Index(['id', 'Basic_Demos-Enroll_Season', 'Basic_Demos-Age', 'Basic_Demos-Sex',
       'CGAS-Season', 'CGAS-CGAS_Score', 'Physical-Season', 'Physical-BMI',
       'Physical-Height', 'Physical-Weight', 'Physical-Waist_Circumference',
       'Physical-Diastolic_BP', 'Physical-HeartRate', 'Physical-Systolic_BP',
       'Fitness_Endurance-Season', 'Fitness_Endurance-Max_Stage',
       'Fitness_Endurance-Time_Mins', 'Fitness_Endurance-Time_Sec',
       'FGC-Season', 'FGC-FGC_CU', 'FGC-FGC_CU_Zone', 'FGC-FGC_GSND',
       'FGC-FGC_GSND_Zone', 'FGC-FGC_GSD', 'FGC-FGC_GSD_Zone', 'FGC-FGC_PU',
       'FGC-FGC_PU_Zone', 'FGC-FGC_SRL', 'FGC-FGC_SRL_Zone', 'FGC-FGC_SRR',
       'FGC-FGC_SRR_Zone', 'FGC-FGC_TL', 'FGC-FGC_TL_Zone', 'BIA-Season',
       'BIA-BIA_Activity_Level_num', 'BIA-BIA_BMC', 'BIA-BIA_BMI',
       'BIA-BIA_BMR', 'BIA-BIA_DEE', 'BIA-BIA_ECW', 'BIA-BIA_FFM',
       'BIA-BIA_FFMI', 'BIA-BIA_FMI', 'BIA-BIA_Fat', 'BIA-BIA_Frame_num',
       'BIA-BIA_ICW', 'BIA-BIA_LDM', 'BIA-BIA_LST'

Merging PAQ children and adolescents columns

In [15]:
idx = df.columns.get_loc('PAQ_A-Season')
df.insert(idx, 'PAQ-PAQ_Total', df.pop('PAQ-PAQ_Total'))
df.columns

Index(['id', 'Basic_Demos-Enroll_Season', 'Basic_Demos-Age', 'Basic_Demos-Sex',
       'CGAS-Season', 'CGAS-CGAS_Score', 'Physical-Season', 'Physical-BMI',
       'Physical-Height', 'Physical-Weight', 'Physical-Waist_Circumference',
       'Physical-Diastolic_BP', 'Physical-HeartRate', 'Physical-Systolic_BP',
       'Fitness_Endurance-Season', 'Fitness_Endurance-Max_Stage',
       'Fitness_Endurance-Time_Mins', 'Fitness_Endurance-Time_Sec',
       'FGC-Season', 'FGC-FGC_CU', 'FGC-FGC_CU_Zone', 'FGC-FGC_GSND',
       'FGC-FGC_GSND_Zone', 'FGC-FGC_GSD', 'FGC-FGC_GSD_Zone', 'FGC-FGC_PU',
       'FGC-FGC_PU_Zone', 'FGC-FGC_SRL', 'FGC-FGC_SRL_Zone', 'FGC-FGC_SRR',
       'FGC-FGC_SRR_Zone', 'FGC-FGC_TL', 'FGC-FGC_TL_Zone', 'BIA-Season',
       'BIA-BIA_Activity_Level_num', 'BIA-BIA_BMC', 'BIA-BIA_BMI',
       'BIA-BIA_BMR', 'BIA-BIA_DEE', 'BIA-BIA_ECW', 'BIA-BIA_FFM',
       'BIA-BIA_FFMI', 'BIA-BIA_FMI', 'BIA-BIA_Fat', 'BIA-BIA_Frame_num',
       'BIA-BIA_ICW', 'BIA-BIA_LDM', 'BIA-BIA_LST'

Removing all columns that contain season except internet usage

In [18]:
col_to_be_kept = 'PreInt_EduHx-Season'
col_names = []
for col in df.columns:
    if 'Season' in col and col != col_to_be_kept:
        col_names.append(col)
print(col_names)
df = df.drop(columns=col_names)
df.columns

['Basic_Demos-Enroll_Season', 'CGAS-Season', 'Physical-Season', 'Fitness_Endurance-Season', 'FGC-Season', 'BIA-Season', 'PAQ_A-Season', 'PAQ_C-Season', 'PCIAT-Season', 'SDS-Season']


Index(['id', 'Basic_Demos-Age', 'Basic_Demos-Sex', 'CGAS-CGAS_Score',
       'Physical-BMI', 'Physical-Height', 'Physical-Weight',
       'Physical-Waist_Circumference', 'Physical-Diastolic_BP',
       'Physical-HeartRate', 'Physical-Systolic_BP',
       'Fitness_Endurance-Max_Stage', 'Fitness_Endurance-Time_Mins',
       'Fitness_Endurance-Time_Sec', 'FGC-FGC_CU', 'FGC-FGC_CU_Zone',
       'FGC-FGC_GSND', 'FGC-FGC_GSND_Zone', 'FGC-FGC_GSD', 'FGC-FGC_GSD_Zone',
       'FGC-FGC_PU', 'FGC-FGC_PU_Zone', 'FGC-FGC_SRL', 'FGC-FGC_SRL_Zone',
       'FGC-FGC_SRR', 'FGC-FGC_SRR_Zone', 'FGC-FGC_TL', 'FGC-FGC_TL_Zone',
       'BIA-BIA_Activity_Level_num', 'BIA-BIA_BMC', 'BIA-BIA_BMI',
       'BIA-BIA_BMR', 'BIA-BIA_DEE', 'BIA-BIA_ECW', 'BIA-BIA_FFM',
       'BIA-BIA_FFMI', 'BIA-BIA_FMI', 'BIA-BIA_Fat', 'BIA-BIA_Frame_num',
       'BIA-BIA_ICW', 'BIA-BIA_LDM', 'BIA-BIA_LST', 'BIA-BIA_SMM',
       'BIA-BIA_TBW', 'PAQ-PAQ_Total', 'PCIAT-PCIAT_01', 'PCIAT-PCIAT_02',
       'PCIAT-PCIAT_03', 'PCIAT-PC

Removing rows that don't have atleast 50% of features available

In [37]:
threshold = df.shape[1]/2
df = df.dropna(thresh=threshold)
df.shape


(2641, 71)

Separating categorical and numerical columns 

In [ ]:
categorical_cols = df.select_dtypes(include=["category", "object"]).columns  
numeric_cols = df.select_dtypes(include=["number"]).columns 

#Converting categorical into numericla data
label_encoder = LabelEncoder()
for col in categorical_cols:
    df[col] = label_encoder.fit_transform(df[col])


0       0
1       2
2       2
3       3
5       1
       ..
3953    0
3954    2
3955    0
3957    0
3958    1
Name: PreInt_EduHx-Season, Length: 2641, dtype: int32


Imputing missing values with KNNImputer

In [50]:
imputer = KNNImputer(n_neighbors=7)
imputed_data = imputer.fit_transform(df[numeric_cols])
train_imputed = pd.DataFrame(imputed_data, columns=numeric_cols) 
train_imputed.head()
if 'sii' in train_imputed.columns:  
    train_imputed['sii'] = train_imputed['sii'].round().astype(int)  
df[numeric_cols] = train_imputed[numeric_cols]
df.head()

C:\Users\serji\AppData\Local\Temp\ipykernel_15020\2979275378.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[numeric_cols] = train_imputed[numeric_cols]


,id,Basic_Demos-Age,Basic_Demos-Sex,CGAS-CGAS_Score,Physical-BMI,Physical-Height,Physical-Weight,Physical-Waist_Circumference,Physical-Diastolic_BP,Physical-HeartRate,...,PCIAT-PCIAT_17,PCIAT-PCIAT_18,PCIAT-PCIAT_19,PCIAT-PCIAT_20,PCIAT-PCIAT_Total,SDS-SDS_Total_Raw,SDS-SDS_Total_T,PreInt_EduHx-Season,PreInt_EduHx-computerinternet_hoursday,sii
0,0.0,5.0,0.0,51.000000,16.877316,46.0,50.8,26.000000,68.0,85.571429,...,4.0,4.0,2.0,4.0,55.0,39.142857,55.428571,0.0,3.0,2.0
1,1.0,9.0,0.0,64.714286,14.035590,48.0,46.0,22.000000,75.0,70.000000,...,0.0,0.0,0.0,0.0,0.0,46.000000,64.000000,2.0,0.0,0.0
2,2.0,10.0,1.0,71.000000,16.648696,56.5,75.6,25.428571,65.0,94.000000,...,2.0,2.0,1.0,1.0,28.0,38.000000,54.000000,2.0,2.0,0.0
3,3.0,9.0,0.0,71.000000,18.292347,56.0,81.6,29.428571,60.0,97.000000,...,4.0,3.0,4.0,1.0,44.0,31.000000,45.000000,3.0,0.0,1.0
5,5.0,10.0,0.0,77.428571,19.660760,55.0,84.6,28.428571,123.0,83.000000,...,1.0,4.0,1.0,0.0,20.0,27.000000,40.000000,0.0,3.0,0.0


In [ ]:
#additional cleaning
df = df.drop('id', axis=1)
df.head()
print(df.shape)

(2641, 70)
